<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=a511f5b4455bf3acd63eb27a261ac1f213a1e7960470e71261791234cbafbfec
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-09 09:28:24
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.42 C
-------------------
Today PnL: -89.79 K (-0.63%)
Current PnL: -27.13 L (-17.3%)
CY Booked + Current PnL: -12.84 L (-8.19%)
-------------------
Total profit:  1.80 L
Total loss:  -28.93 L
-------------------
Total Booked + Current PnL: 14.14 L (10.89%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.07%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 92.58 L (65.22%)
Deployed:  1.30 C
Current:  1.42 C
CAGR/XIRR %: 4.87%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,-0.05,15.15,5.42,21.40,16566.0,40225.0,305651.0,6.83,63.0,X-LC,5.73,14.0,2.43,2.15,22.44,XY25,NTT,FMCG
77,TTKPRESTIG,770.00,-0.19,-21.55,27.59,0.09,21812.0,-21719.0,79058.0,83.56,34.0,M-SC,3.32,253.0,-1.00,0.56,3.87,OX40N,NTT,DURABLES
17,COALINDIA,484.83,-0.15,4.41,14.19,19.23,22293.0,6642.0,157102.0,22.81,73.0,L-LC,7.12,182.0,0.30,1.11,26.63,XY25,ATH,MINING
49,MASFIN,398.61,-1.07,-2.69,25.43,22.05,24245.0,-2640.0,95340.0,-15.99,49.0,H-SC,6.56,168.0,-0.11,0.67,38.58,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-1.52,-14.59,17.10,0.02,25496.0,-25461.0,149098.0,127.12,53.0,M-SC,8.06,240.0,-1.00,1.05,29.69,OX40N,NTT,PAINTS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,IEX,219.00,1.61,3.49,43.61,48.63,90247.0,6988.0,206942.0,-31.26,65.0,H-SC,13.31,137.0,0.08,1.46,15.52,XR,NTT,MISC
44,JCHAC,2282.00,1.36,-33.96,51.40,-0.01,43386.0,-43396.0,84409.0,17182.61,52.0,M-SC,13.73,235.0,-1.00,0.59,8.03,OX40N,BTT,AC
5,ASIANPAINT,3460.25,1.27,-1.37,22.62,20.95,56172.0,-3441.0,248327.0,-2.32,54.0,X-LC,12.24,36.0,-0.06,1.75,33.78,X40,ATH,PAINTS
19,COLPAL,3726.84,1.13,-21.41,80.07,41.51,165721.0,-56395.0,206970.0,-6.31,41.0,X-MC,9.08,65.0,-0.34,1.46,1.13,XY25,ATH,FMCG
65,SIEMENS,4671.50,1.05,-18.29,53.61,25.51,81519.0,-34035.0,152060.0,-6.67,42.0,H-LC,1.76,50.0,-0.42,1.07,12.06,AR,ATH,ELECTRICAL


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,7934.00,-5.25,-7.88,58.02,45.57,139833.0,-20611.0,241008.0,-40.82,40.0,H-MC,4.66,121.0,-0.15,1.70,2.55,AR,NTT,ELECTRICAL
76,TRIDENT,48.00,-3.14,-29.69,82.79,28.51,53699.0,-27392.0,64862.0,-16.89,39.0,M-SC,7.41,226.0,-0.51,0.46,10.99,XR,NTT,TEXTILES
61,ROUTE,2227.21,-2.65,-48.73,229.74,69.05,156729.0,-64846.0,68220.0,-58.67,42.0,H-SC,18.88,140.0,-0.41,0.48,5.21,SR,ATH,IT
54,QUESS,424.00,-2.50,-28.36,99.40,42.84,46289.0,-18438.0,46568.0,-53.17,49.0,X-SC,27.05,83.0,-0.40,0.33,5.36,XY24,NTT,MISC
4,ALKYLAMINE,4546.37,-2.50,-33.25,190.08,93.63,128103.0,-33569.0,67394.0,-28.56,42.0,H-SC,16.62,143.0,-0.26,0.47,1.89,SR,ATH,CHEMICALS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.0,-0.7,-0.16,20.93,20.74,42454.0,-330.0,202840.0,31.15,56.0,M-MC,5.39,189.0,-0.01,1.43,10.44,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.08,-1.68,111.46,107.91,161804.0,-2476.0,145168.0,-20.15,54.0,M-SC,10.10,220.0,-0.02,1.02,8.24,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.52,-14.59,17.10,0.02,25496.0,-25461.0,149098.0,127.12,53.0,M-SC,8.06,240.0,-1.00,1.05,29.69,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,-2.33,-17.61,60.26,32.03,65451.0,-23221.0,108614.0,-10.83,66.0,H-SC,6.96,157.0,-0.35,0.77,21.66,OX40N,NTT,IT
77,TTKPRESTIG,770.00,-0.19,-21.55,27.59,0.09,21812.0,-21719.0,79058.0,83.56,34.0,M-SC,3.32,253.0,-1.00,0.56,3.87,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.78,-22.32,46.08,13.47,96526.0,-60192.0,209475.0,-68.12,47.0,H-SC,4.60,159.0,-0.62,1.48,6.51,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,-0.15,4.41,14.19,19.23,22293.0,6642.0,157102.0,22.81,73.0,L-LC,7.12,182.0,0.30,1.11,26.63,XY25,ATH,MINING
84,WIPRO,420.00,-0.44,7.07,60.89,72.27,110473.0,11982.0,181430.0,-7.89,48.0,M-LC,6.15,101.0,0.11,1.28,13.68,XR,NTT,IT
37,IEX,219.00,1.61,3.49,43.61,48.63,90247.0,6988.0,206942.0,-31.26,65.0,H-SC,13.31,137.0,0.08,1.46,15.52,XR,NTT,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.44,7.07,60.89,72.27,110473.0,11982.0,181430.0,-7.89,48.0,M-LC,6.15,101.0,0.11,1.28,13.68,XR,NTT,IT
37,IEX,219.00,1.61,3.49,43.61,48.63,90247.0,6988.0,206942.0,-31.26,65.0,H-SC,13.31,137.0,0.08,1.46,15.52,XR,NTT,MISC
25,FINCABLES,1641.55,0.08,-1.68,111.46,107.91,161804.0,-2476.0,145168.0,-20.15,54.0,M-SC,10.10,220.0,-0.02,1.02,8.24,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-0.74,-7.57,123.66,106.72,140970.0,-9338.0,113998.0,-54.25,35.0,H-SC,7.38,138.0,-0.07,0.80,14.04,AR,ATH,MISC
49,MASFIN,398.61,-1.07,-2.69,25.43,22.05,24245.0,-2640.0,95340.0,-15.99,49.0,H-SC,6.56,168.0,-0.11,0.67,38.58,XR,ATH,FINANCE


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.31,-16.71,33.00,10.78,66057.0,-40146.0,200172.0,-50.55,13.0,X-LC,6.27,1.0,-0.61,1.41,0.00,X40,NTT,FMCG
7,AWL,485.00,-1.27,-27.09,115.60,57.19,274864.0,-88355.0,237772.0,-65.54,25.0,X-MC,9.24,58.0,-0.32,1.68,0.00,XY24,NTT,FMCG
52,PAGEIND,51605.08,-0.43,-12.39,49.08,30.62,101934.0,-29364.0,207690.0,-37.71,27.0,X-MC,16.40,57.0,-0.29,1.46,0.00,X40,ATH,APPARELS
11,BATAINDIA,2096.00,-1.05,-40.82,128.46,35.20,97820.0,-52522.0,76148.0,-0.22,29.0,X-SC,17.37,93.0,-0.54,0.54,0.00,X40,NTT,FOOTWEAR
59,RELIANCE,1952.00,-2.23,-5.60,32.73,25.30,62091.0,-11256.0,189707.0,-8.81,32.0,X-LC,8.44,26.0,-0.18,1.34,27.07,XY25,BTT,REFINERIES


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,1644.00,-0.12,-0.93,21.90,20.77,50799.0,-2185.0,231959.0,-9.57,33.0,X-MC,0.95,70.0,-0.04,1.63,5.92,X40N,NTT,BREWERIES
45,JIOFIN,387.00,-0.49,-6.04,32.49,24.48,81617.0,-16159.0,251206.0,-7.58,42.0,X-LC,1.72,37.0,-0.20,1.77,45.61,XY24,BTT,FINANCE
65,SIEMENS,4671.50,1.05,-18.29,53.61,25.51,81519.0,-34035.0,152060.0,-6.67,42.0,H-LC,1.76,50.0,-0.42,1.07,12.06,AR,ATH,ELECTRICAL
20,DABUR,735.00,0.23,1.79,41.17,43.70,103960.0,4447.0,252515.0,-6.68,65.0,X-MC,2.24,73.0,0.04,1.78,17.11,XY24,BTT,FMCG
81,VBL,671.64,-0.37,0.81,34.58,35.67,110101.0,2552.0,318394.0,-11.24,61.0,X-LC,2.26,5.0,0.02,2.24,14.77,X40N,ATH,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.53,-27.96,127.86,64.15,219178.0,-66531.0,171420.0,-58.06,35.0,X-SC,6.26,84.0,-0.30,1.21,0.0,XY24,BTT,CEMENT
7,AWL,485.00,-1.27,-27.09,115.60,57.19,274864.0,-88355.0,237772.0,-65.54,25.0,X-MC,9.24,58.0,-0.32,1.68,0.0,XY24,NTT,FMCG
11,BATAINDIA,2096.00,-1.05,-40.82,128.46,35.20,97820.0,-52522.0,76148.0,-0.22,29.0,X-SC,17.37,93.0,-0.54,0.54,0.0,X40,NTT,FOOTWEAR
8,BAJAJHFL,181.50,-1.46,-19.89,94.08,55.47,183358.0,-48390.0,194896.0,-29.11,33.0,X-MC,16.34,63.0,-0.26,1.37,0.0,X40N,ATH,FINANCE
52,PAGEIND,51605.08,-0.43,-12.39,49.08,30.62,101934.0,-29364.0,207690.0,-37.71,27.0,X-MC,16.40,57.0,-0.29,1.46,0.0,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-2.50,-28.36,99.40,42.84,46289.0,-18438.0,46568.0,-53.17,49.0,X-SC,27.05,83.0,-0.40,0.33,5.36,XY24,NTT,MISC
58,RELAXO,1176.00,-1.14,-48.80,197.46,52.31,146764.0,-70834.0,74326.0,-45.50,36.0,X-SC,7.86,92.0,-0.48,0.52,0.00,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.05,-40.82,128.46,35.20,97820.0,-52522.0,76148.0,-0.22,29.0,X-SC,17.37,93.0,-0.54,0.54,0.00,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,-1.73,-1.46,25.91,24.08,32414.0,-1849.0,125101.0,-9.50,47.0,X-SC,6.29,91.0,-0.06,0.88,17.87,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.87,-17.89,74.54,43.32,99690.0,-29132.0,133740.0,-29.42,42.0,X-SC,7.87,90.0,-0.29,0.94,2.69,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.31,-16.71,33.00,10.78,66057.0,-40146.0,200172.0,-50.55,13.0,X-LC,6.27,1.0,-0.61,1.41,0.00,X40,NTT,FMCG
75,TMPV,600.00,-0.85,-26.95,68.35,22.98,112786.0,-60885.0,165013.0,-24.30,43.0,X-LC,4.11,2.0,-0.54,1.16,3.79,XY24,NTT,AUTO
73,TCS,4389.97,-0.05,-11.95,37.08,20.71,123495.0,-45187.0,333050.0,-24.27,47.0,X-LC,2.48,3.0,-0.37,2.35,11.28,X40,ATH,IT
81,VBL,671.64,-0.37,0.81,34.58,35.67,110101.0,2552.0,318394.0,-11.24,61.0,X-LC,2.26,5.0,0.02,2.24,14.77,X40N,ATH,FMCG
41,INFY,1972.00,0.21,10.06,21.98,34.24,75690.0,31469.0,344357.0,-14.21,51.0,X-LC,7.68,6.0,0.42,2.43,19.17,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.03,-34.75,103.68,32.89,52228.0,-26832.0,50374.0,-721.83,58.0,L-MC,7.73,259.0,-0.51,0.35,38.73,XR,NTT,BANKS
49,MASFIN,398.61,-1.07,-2.69,25.43,22.05,24245.0,-2640.0,95340.0,-15.99,49.0,H-SC,6.56,168.0,-0.11,0.67,38.58,XR,ATH,FINANCE
13,BSOFT,831.70,-1.18,-22.67,94.98,50.79,103310.0,-31883.0,108770.0,-0.66,50.0,H-SC,10.67,171.0,-0.31,0.77,26.64,XR,ATH,IT
69,SURYODAY,216.00,-0.03,-17.44,49.02,23.03,72476.0,-31222.0,147849.0,60.09,59.0,H-SC,4.14,165.0,-0.43,1.04,46.70,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,-1.52,-14.59,17.10,0.02,25496.0,-25461.0,149098.0,127.12,53.0,M-SC,8.06,240.0,-1.00,1.05,29.69,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,0.51,-28.49,57.88,12.90,90438.0,-62246.0,156250.0,130.77,65.0,H-SC,10.20,173.0,-0.69,1.10,29.92,OX40N,NTT,FINANCE
72,TATAELXSI,9161.00,-2.33,-17.61,60.26,32.03,65451.0,-23221.0,108614.0,-10.83,66.0,H-SC,6.96,157.0,-0.35,0.77,21.66,OX40N,NTT,IT
17,COALINDIA,484.83,-0.15,4.41,14.19,19.23,22293.0,6642.0,157102.0,22.81,73.0,L-LC,7.12,182.0,0.30,1.11,26.63,XY25,ATH,MINING
82,VOLTAS,1856.00,0.31,15.28,25.95,45.20,57360.0,29298.0,221040.0,4.03,63.0,X-MC,4.93,80.0,0.51,1.56,23.12,XY25,NTT,AC
18,COFFEEDAY,80.00,-0.09,-41.00,129.29,35.27,86610.0,-46560.0,66989.0,-55.73,46.0,L-SC,3.98,268.0,-0.54,0.47,63.96,XR,NTT,HOTELS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.34
1,25,44.36
2,50,76.47


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.80
MC    30.68
LC    26.52
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.31
X40      23.94
X40N     12.78
XY25     10.73
XR        9.08
AR        8.52
OX40N     7.69
SR        0.95
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.30
X-LC    23.06
H-SC    22.91
M-SC    11.28
X-SC     7.82
H-MC     4.60
M-MC     1.43
M-LC     1.28
L-LC     1.11
H-LC     1.07
L-SC     0.79
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.10,-8.53,44.14
IT,12.84,-17.38,76.59
FINANCE,11.10,-13.21,60.72
MISC,7.20,-27.00,78.95
ELECTRICAL,5.92,-11.74,52.66
PAINTS,5.74,-13.33,30.16
INSURANCE,4.69,-1.83,36.68
PHARMA,3.92,-5.57,38.80
AUTO,2.77,-33.16,79.33


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3237425.0,21
AR,1362158.0,10
XR,1203379.0,12
X40,1124272.0,15
X40N,923629.0,9
OX40N,701715.0,10
XY25,420891.0,7
SR,284832.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3543208.0,24
X-MC,1582024.0,16
M-SC,1458665.0,15
X-LC,951278.0,13
X-SC,809415.0,8
H-MC,426416.0,3
L-SC,178328.0,2
M-LC,110473.0,1
H-LC,81519.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1257385.0      6
           AR         923961.0      5
M-SC       XY24       840533.0      6
H-SC       XR         770396.0      7
X-MC       X40        510034.0      7
           XY24       446388.0      3
X-LC       X40        417966.0      6
X-MC       X40N       402521.0      4
X-SC       X40N       315262.0      3
H-SC       OX40N      306634.0      4
M-SC       OX40N      303363.0      5
X-SC       XY24       297881.0      3
H-SC       SR         284832.0      2
H-MC       AR         225581.0      2
X-MC       XY25       223081.0      2
X-LC       X40N       205846.0      2
H-MC       XY24       200835.0      1
X-SC       X40        196272.0      2
X-LC       XY24       194403.0      2
M-SC       XR         183672.0      2
X-LC       XY25       133063.0      3
M-SC       AR         131097.0      2
M-LC       XR         110473.0      1
L-SC       OX40N       91718.0      1
           XR          86610.0      1
H-LC       AR          81519.0      1
L-MC       XR          52228.0      1
M-MC       XY25        42454.0      1
L-LC       XY25        22293.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
